# 셀을 실행할 때 팝업이 뜨면 `무시하고 계속하기` 를 눌러주세요


## 한글 글꼴 다운로드
---
해당 셀을 실행하고 [런타임] - [런타임 다시 시작] 후 아래 셀들을 수행하세요.

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

## 데이터 다운로드
---
아래 셀을 실행시켜 데이터를 colab 에 불러옵니다.
셀 실행은 Ctrl + Enter 를 이용해 실행시킬 수 있습니다.

In [ ]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

## 라이브러리 불러오기 및 설정 세팅
---

In [ ]:
import pandas as pd
import numpy as np

# 시각화에 필요한 라이브러리를 import
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리를 위한 라이브러리를 import
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# 모델링을 위한 라이브러리를 import
from sklearn.ensemble import RandomForestClassifier

# 교차검증을 위한 라이브러리를 import
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score

# 쥬피터노트에서 결과를 출력하도록 설정
%matplotlib inline

# 글꼴 설정
plt.rc('font', family='NanumBarunGothic')

# 레티나 디스플레이 설정
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False)

# 분석에 문제가 없는 경고 메세지는 숨긴다.
import warnings
warnings.filterwarnings('ignore')

## 👋 실습
---

### EDA

In [ ]:
# read_csv() 매서드로 train.csv, test.csv 파일을 df class 로 불러오세요.
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [ ]:
# info() 매서드로 데이터의 정보를 확인하세요.
train.info()

In [ ]:
# shape 어트리뷰트로 행, 열을 파악하세요.
print(train.shape)
print(test.shape)

In [ ]:
# head() 매서드로 데이터의 각 컬럼의 정보를 조사하세요.
train.head()

In [ ]:
# isnull() 매서드로  결측치 존재여부를 확인하세요.
train.isnull()

In [ ]:
# sum() 매서드로 결측치의 갯수를 출력하세요.
train.isnull().sum()

In [ ]:
# copy() 매서드로 학습 데이터의 복사본을 생성하세요.
traincopy = train.copy()
traincopy

In [ ]:
# 타깃 변수(와인품질)의 분포를 시각화합니다.

#1 seaborn 의 displot() 으로 어느 변수를 시각화할지 옵션에 지정해주세요.
sns.distplot(traincopy['quality'], kde=False, bins=10)
#2 matplot 의 axis() 로 각 축의 최소, 최대 값을 지정해주세요.
plt.axis([0, 10, 0, 2500])
#3 matplot 의 title() 로 그래프의 제목을 지정해주세요.
plt.title("와인 품질")
#4 matplot 의 show() 로 그래프를 출력해주세요.
plt.show()

In [ ]:
# x축 지점의 값들로 정할 리스트를 생성합니다.
x_values = [0, 1, 2, 3, 4]

In [ ]:
# y축 지점의 값들로 정할 리스트를 생성합니다.
y_values = [0, 1, 4, 9, 16]

In [ ]:
# plot() 매서드를 활용해 그래프를 그린 후 화면에 보여줍니다.
plt.plot(x_values, y_values)
plt.show()

In [ ]:
# 변수 분포를 갖는 리스트를 생성합니다.
a = [1,2,3,3,3,4,4,4,4,5,5,5,5,5,6,6,7]

In [ ]:
# plot() 매서드를 활용해 그래프를 그린 후 화면에 보여줍니다.
plt.hist(a)
plt.show()

### 전처리

In [ ]:
# boxplot() 매서드로 'fixed acidity' 피쳐의 이상치를 확인하는 코드를 아래에 작성하세요.
sns.boxplot(data=train['fixed acidity'])

In [ ]:
# "fixed acidity"가 25%인 값을 "quantile_25" 라는 변수에 만들어 주세요.
quantile_25 = np.quantile(train['fixed acidity'], 0.25)

In [ ]:
# "fixed acidity"가 75%인 값을 "quantile_75" 라는 변수에 만들어 주세요.
quantile_75 = np.quantile(train['fixed acidity'], 0.75)

In [ ]:
# quantile_75와 quantile_25의 차이를 "IQR"이라는 변수에 만들어 주세요.
IQR = quantile_75 - quantile_25

In [ ]:
# quantile_25보다 1.5 * IQR 작은 값을 "minimum"이라는 변수에 만들어 주세요.
minimum = quantile_25 - 1.5*IQR

In [ ]:
# quantile_75보다 1.5 * IQR 큰 값을 "maximum"이라는 변수에 만들어 주세요.
maximum = quantile_75 + 1.5*IQR

In [ ]:
# "fixed acidity"가 minimum보다 크고, maximum보다 작은 값들만 "train2"에 저장해 주세요.
train2 = train[(minimum<=train['fixed acidity'])&(train['fixed acidity']<=maximum)]

In [ ]:
# train2.shape를 통해서, 총 몇개의 행이 되었는지 확인해보세요.
train2.shape

In [ ]:
# 몇개의 이상치가 있는지도 계산해보세요.
train.shape[0] - train2.shape[0]

In [ ]:
# describe를 통해 "fixed acidity"의 데이터의 분포가 어떻게 생겼는지 짐작해보세요.
train['fixed acidity'].describe()

In [ ]:
# seaborn의 displot을 통해 "fixed acidity"의 distplot을 그려보세요.
sns.distplot(train['fixed acidity'])

In [ ]:
# MinMaxScaler를 "scaler"라는 변수에 지정해주세요.
scaler = MinMaxScaler()

In [ ]:
# "scaler"를 학습시켜주세요.
scaler.fit(train[['fixed acidity']])

In [ ]:
# "scaler"를 통해 train과 test의 "fixed acidity"를 바꾸어 "Scaled fixed acidity"라는 column에 저장해주세요.
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

In [ ]:
# seaborn의 displot을 통해 "Scaled fixed acidity"의 distplot을 그려보세요.
sns.displot(train['Scaled fixed acidity'])

In [ ]:
# "OneHotEncoder"를 "encoder"라는 변수에 저장해보세요.
encoder = OneHotEncoder()

In [ ]:
# "encoder"를 사용해 train의 "type" 피쳐를 학습시켜보세요.
encoder.fit(train[['type']])

In [ ]:
# "encoder"를 사용해 train의 "type"피쳐를 변환해 "onehot"이라는 변수에, test의 "type"피쳐를 변환해 "onehot2"라는 변수에 저장해보세요.
onehot = encoder.transform(train[['type']])
onehot2 = encoder.transform(test[['type']])

In [ ]:
# "onehot", "onehot2" 라는 변수를 array 형태로 변환해 보세요.
onehot = onehot.toarray()
onehot2 = onehot2.toarray()

In [ ]:
# "onehot","onethot2"라는 변수를 DataFrame 형태로 변환해 보세요. 
onehot = pd.DataFrame(onehot)
onehot2 = pd.DataFrame(onehot2)

In [ ]:
# encoder의 "get_feature_names()"를 사용해 column 이름을 바꿔보세요.
onehot.columns = encoder.get_feature_names()
onehot.head()
onehot2.columns = encoder.get_feature_names()
onehot2.head()

In [ ]:
# onehot, onehot2를 원본데이터인 train,test에 병합시켜보세요.
onehot = pd.concat([train, onehot], axis = 1)
display(onehot.head())
onehot2 = pd.concat([test, onehot2], axis = 1)
display(onehot2.head())

In [ ]:
# train과 test의 "type" 변수를 제거해주세요.
train = train.drop(columns = ['type'])
test = test.drop(columns = ['type'])

### 모델링

In [ ]:
# 랜덤포레스트 분류 모형을 불러오세요.
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# 랜덤포레스트 분류 모형을 "random_classifier"라는 변수에 저장하세요.
random_classifier = RandomForestClassifier()

In [ ]:
# "X"라는 변수에 train의 "quality" 피쳐를 제거하고 저장하세요.
X = train.drop(columns=['quality'])
X.head()

In [ ]:
# "y"라는 변수에 정답인 train의 "quality" 피쳐를 저장하세요.
y = train['quality']
y.head()

In [ ]:
# "random_classifier"를 X와 y를 이용해 학습시켜보세요.
random_classifier.fit(X, y)

In [ ]:
# sklearn에 model_selection 부분 속 KFold를 불러와보세요.
from sklearn.model_selection import KFold

In [ ]:
# KFold에 n_splits = 5, shuffle = True, random_state = 0이라는 인자를 추가해 "kf"라는 변수에 저장해보세요.
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
# 반복문을 통해서 1번부터 5번까지의 데이터에 접근해보세요.
for train_idx, valid_idx in kf.split(train):
  train_data = train.iloc[train_idx]
  valid_data = train.iloc[valid_idx]

In [ ]:
# "X"라는 변수에 train의 "index"와 "quality"를 제외하고 지정해 주세요.
# "y"라는 변수에는 "quality"를 지정해 주세요.
X = train.drop(columns=['index', 'quality'])
y = train['quality']

In [ ]:
# "kf"라는 변수에 KFold를 지정해 줍시다.
# n_splits는 5, shuffle은 True, random_state는 0으로 설정해주세요.
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
# "model"이라는 변수에 RandomForestClassifier를 지정해 줍시다.
# valid_scores라는 빈 리스트를 하나 만들어줍시다.
# test_predictions라는 빈 리스트를 하나 만들어 줍시다.
model = RandomForestClassifier()
valid_scores = []
test_predictions = []

In [ ]:
# 지난 시간에 다루었던 kf.split()을 활용해, 반복문으로 X_tr, y_tr, X_val, y_val을 설정해봅시다.
for train_idx, valid_idx in kf.split(X,y):
  X_tr = X.iloc[train_idx]
  y_tr = y.iloc[train_idx]
  X_val = X.iloc[valid_idx]
  y_val = y.iloc[valid_idx]

In [ ]:
# 앞의 문제에 이어서 반복문 속에서 model.fit(X_tr, y_tr)을 활용해 모델을 학습해봅시다.
for train_idx, valid_idx in kf.split(X, y):
  X_tr = X.iloc[train_idx]
  y_tr = y.iloc[train_idx]
  X_val = X.iloc[valid_idx]
  y_val = y.iloc[valid_idx]
  model.fit(X_tr, y_tr)

In [ ]:
# 앞의 문제에 이어서 반복문 속에서 "valid_prediction"이라는 변수에 model.predict(X_val)의 결과를 저장해 봅시다. 
for train_idx, valid_idx in kf.split(X, y):
  X_tr = X.iloc[train_idx]
  y_tr = y.iloc[train_idx]
  X_val = X.iloc[valid_idx]
  y_val = y.iloc[valid_idx]
  model.fit(X_tr, y_tr)
  valid_prediction = model.predict(X_val)

In [ ]:
# 앞의 문제에 이어서 반복문 속에서 accuracy_score를 이용해, 모델이 어느정도의 예측 성능이 나올지 확인해봅시다.
# 그리고 "valid_prediction"의 점수를 scores에 저장 해봅시다. 
# 반복문에서 빠져나온 후에 np.mean()을 활용해 평균 점수를 예측해봅시다.
for train_idx, valid_idx in kf.split(X, y):
  X_tr = X.iloc[train_idx]
  y_tr = y.iloc[train_idx]
  X_val = X.iloc[valid_idx]
  y_val = y.iloc[valid_idx]
  model.fit(X_tr, y_tr)
  valid_prediction = model.predict(X_val)
  score = accuracy_score(y_val, valid_prediction)
  valid_scores.append(score)
  print(score)
print('평균 점수:', np.mean(valid_scores))

In [ ]:
# 이제 어느정도의 성능이 나올지 알게 되었으니, 반복문 속에서 test를 예측해 "test_prediction"이라는 변수에 지정해봅시다.
# test_prediction을 지정했다면, "test_precitions"라는 빈 리스트에 넣어줍시다.
for train_idx, valid_idx in kf.split(X, y):
  X_tr = X.iloc[train_idx]
  y_tr = y.iloc[train_idx]
  X_val = X.iloc[valid_idx]
  y_val = y.iloc[valid_idx]
  model.fit(X_tr, y_tr)
  test_prediction = model.predict(test.drop(columns=['index']))
  test_predictions.append(test_prediction)

In [ ]:
# 이제 결과 값을 만들어 보겠습니다.
# "test_precitions"를 Data Frame으로 만들어주세요.
test_predictions = pd.DataFrame(test_predictions)
test_predictions.head()

In [ ]:
# DF.mode()를 활용해 열별 최빈값을 확인하고, "test_prediction"이라는 변수에 지정해봅시다.
# "test_prediction"의 첫 행을 최종 결과값으로 사용합시다.
test_prediction = test_predictions.mode()
test_prediction = test_predictions.values[0]

In [ ]:
# data의 sample_submission 파일을 불러와 "quality"라는 변수에 "test_precition"을 저장해줍시다.
# 그 이후에는, "data/submission_KFOLD.csv"에 저장하고, 제출해봅시다.
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission["quality"] = test_prediction
sample_submission.to_csv("data/submission_KFOLD.csv", index=False)

### 튜닝

In [ ]:
# bayesian-optimization을 설치해보세요.
!pip install bayesian-optimization

In [ ]:
# bayes_opt 패키지에서 BayesianOptimization을 불러와보세요.
from bayes_opt import BayesianOptimization

In [ ]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요
X = train.drop(columns=['index', 'quality'])
y = train['quality']

In [ ]:
# 랜덤포레스트의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요
## Key는 랜덤포레스트의 hyperparameter이름이고, value는 탐색할 범위 입니다.
rf_parameter_bounds = {
          'max_depth':(1,3),
          'n_estimators':(20,100),
}

In [ ]:
# 함수를 만들어주겠습니다.
# 함수의 구성은 다음과 같습니다.
# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들
# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성
# 3. 그 딕셔너리를 바탕으로 모델 생성
# 4. train_test_split을 통해 데이터 train-valid 나누기
# 5 .모델 학습
# 6. 모델 성능 측정
# 7. 모델의 점수 반환
def rf_bo(max_depth, n_estimators):
  rf_params = {
              'max_depth' : int(round(max_depth)),
               'n_estimators' : int(round(n_estimators)),      
              }
  rf = RandomForestClassifier(**rf_params)

  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, )

  rf.fit(X_train,y_train)
  score = accuracy_score(y_valid, rf.predict(X_valid))
  return score

In [ ]:
# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.
# "BO_rf"라는 변수에 Bayesian Optmization을 저장해보세요
BO_rf = BayesianOptimization(f=rf_bo, pbounds=rf_parameter_bounds, random_state=0)

In [ ]:
# Bayesian Optimization을 실행해보세요
BO_rf.maximize(init_points=5, n_iter=5)

In [ ]:
# 하이퍼파라미터의 결과값을 불러와 "max_params"라는 변수에 저장해보세요
max_params = BO_rf.max['params']
max_params['max_depth'] = int(max_params['max_depth'])
max_params['n_estimators'] = int(max_params['n_estimators'])
print(max_params)

In [ ]:
# Bayesian Optimization의 결과를 "BO_tuend_rf"라는 변수에 저장해보세요
BO_tuend_rf = RandomForestClassifier(**max_params)